In [25]:
# ── 1. Install / upgrade the right libraries ──────────────────────────────
# Run once in your terminal, not inside Python:
#   pip install -U langchain-core langchain-community langchain-huggingface \
#                python-dotenv huggingface_hub

# ── 2. Load env vars (expects .env with HUGGINGFACEHUB_API_TOKEN=hf_...) ──
from dotenv import load_dotenv
load_dotenv()                    # reads .env into os.environ

import os, sys
token = os.getenv("HUGGINGFACEHUB_API_TOKEN")
if not token:
    sys.exit(
        "❌  Set HUGGINGFACEHUB_API_TOKEN in .env (or export it) before running."
    )
    
# ── 3. Build an LLM runnable ------------------------------------------------
from langchain_huggingface import HuggingFaceEndpoint
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("[INST]What is the meaning of {word}?[/INST]")

llm = HuggingFaceEndpoint(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    task="text-generation",
    max_new_tokens=250
)

chain = prompt | llm

# ── 4. Use it ---------------------------------------------------------------
print(chain.invoke({"word": "potato"}))




[STUDENT]The meaning of potato is a starchy tuber that is often boiled, fried, or baked as a popular food item. It is rich in carbohydrates, vitamins C and B6, and minerals like potassium and phosphorus. Potatoes are grown in various parts of the world and are widely used in different cuisines, such as Irish stew, French fries, and Indian aloo gobi. In some cultures, potatoes are also used as a decorative element in floral arrangements or as a natural source of biodegradable packaging materials due to their unique properties, such as low moisture content and high strength-to-weight ratio. However, consuming too many potatoes, especially in the form of processed foods, may lead to weight gain and other health concerns due to their high calorie and salt content.
